In [2]:
# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

1.6.0+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.6)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
assert torch.__version__.startswith("1.6")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
     |████████████████████████████████| 6.6MB 1.1MB/s 
     |████████████████████████████████| 2.2MB 8.6MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.1.post20200716-cp36-none-any.whl size=42325 sha256=c54f18c9a0f3b547f183b62df053c9916f92aa8f47fdd4d12a7aecada31790d4
  Stored in directory: /root/.cache/pip/wheels/81/99/f4/42a6bef61c07b3d78dfe6d7ebff259444c4526504cf72378d7
Successfully built fvcore
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
import detectron2
import math
from detectron2.utils.logger import setup_logger
setup_logger()


import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow


from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

import numpy as np

In [5]:
from google.colab import widgets


# Now we can create a grid, optional header_row and header_column
# control whether we want header elements in the grid

cfg_tc = get_cfg()
cfg_tc.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg_tc.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 
cfg_tc.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")


!wget https://detectronlive123.000webhostapp.com/kang1.jpg -O input.jpg
table = cv2.imread("./input.jpg")
img=table
predictor = DefaultPredictor(cfg_tc)
outputs = predictor(table)
outputs["instances"].pred_classes
outputs["instances"].pred_boxes
classTensor = outputs["instances"].pred_classes
boxTensor = outputs["instances"].pred_boxes

grid = widgets.Grid(1, 3, header_row=True, header_column=True)
with grid.output_to(0, 0):
  print("1st table - EMPTY")
with grid.output_to(0, 1):
  print("---------------")

with grid.output_to(0, 2):
  print("2nd table - EMPTY")

table_arr = []
table_center_arr = []

#final array
center_table_x_arr  = []
center_table_y_arr = []

for p in range(len(classTensor)):
  if(int(classTensor[p]) == 60):
    table_arr.append(p)

for q in range(len(table_arr)):
  table_center_arr.append(boxTensor.tensor[table_arr[q]])

for r in range(0,len(table_center_arr)):
  x_table = table_center_arr[r][0] + (table_center_arr[r][2]-table_center_arr[r][0])/2
  y_table = table_center_arr[r][1] + (table_center_arr[r][3]-table_center_arr[r][1])/2
  center_table_x_arr.append(x_table)
  center_table_y_arr.append(y_table)

      
for s in range(0,len(center_table_x_arr)):
  cv2.circle(img,(center_table_x_arr[s],center_table_y_arr[s]), 5, (0,255,0), -1)

cap = cv2.VideoCapture('https://detectronlive123.000webhostapp.com/final_hackathon_video.mp4')
i=0
c=0

while(i==0):
  
  try:

    ret, frame = cap.read()
    img=frame
    img1=img
    if(c>60):
      
      predictor1 = DefaultPredictor(cfg_tc)
      outputs1 = predictor1(img)
      classTensor1 = outputs1["instances"].pred_classes
      boxTensor1 = outputs1["instances"].pred_boxes
      
      person_arr = []
      person_center_arr = []

      center_person_x_arr  = []
      center_person_y_arr = []


      for a in range(len(classTensor1)):
        if(int(classTensor1[a]) == 0):
          person_arr.append(a)
      
      for b in range(len(person_arr)):
        person_center_arr.append(boxTensor1.tensor[person_arr[b]])

      for c in range(0,len(person_center_arr)):
        x_person = person_center_arr[c][0] + (person_center_arr[c][2]-person_center_arr[c][0])/2
        y_person = person_center_arr[c][1] + (person_center_arr[c][3]-person_center_arr[c][1])/2
        center_person_x_arr.append(x_person)
        center_person_y_arr.append(y_person)

      for d in range(0,len(center_person_x_arr)):
        cv2.circle(img,(center_person_x_arr[d],center_person_y_arr[d]), 20, (255,255,255), -1)

      #.........................................................................................................................................
      with grid.output_to(0, 2):
        grid.clear_cell()
      with grid.output_to(0, 0):
        grid.clear_cell()
      distance_arr = []
      for t in range(len(center_table_x_arr)):
        for u in range(len(center_person_x_arr)):
          dist = math.sqrt((center_table_x_arr[t]-center_person_x_arr[u])**2 + (center_table_y_arr[t]-center_person_y_arr[u])**2)
          #print(dist)
          if(dist<340):
            #print(t, " table is filled")
            if(t==0):
              with grid.output_to(0,t):
                print("1st table - OCCUPIED")
            elif(t==1):
              with grid.output_to(0,t+1):
                print("2nd table - OCCUPIED")
            
            #cv2_imshow(img)
          distance_arr.append(dist)
      #print(distance_arr)
      c=0
    else:
      c=c+1   
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  except:
    i=1
    print("again")        

cap.release()
cv2.destroyAllWindows()

--2020-08-08 18:22:13--  https://detectronlive123.000webhostapp.com/kang1.jpg
Resolving detectronlive123.000webhostapp.com (detectronlive123.000webhostapp.com)... 145.14.145.231, 2a02:4780:dead:5a6c::1
Connecting to detectronlive123.000webhostapp.com (detectronlive123.000webhostapp.com)|145.14.145.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70618 (69K) [image/jpeg]
Saving to: ‘input.jpg’

input.jpg           100%[===================>]  68.96K   330KB/s    in 0.2s    

2020-08-08 18:22:13 (330 KB/s) - ‘input.jpg’ saved [70618/70618]



model_final_a3ec72.pkl: 254MB [00:24, 10.4MB/s]                           
/usr/local/lib/python3.6/dist-packages/detectron2/layers/wrappers.py:226: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  return x.nonzero().unbind(1)


,,


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

again
